## எம்பெடிங்ஸ்

முந்தைய உதாரணத்தில், `vocab_size` நீளமுள்ள உயர்-பரிமாண bag-of-words வெக்டர்களில் செயல்பட்டோம், மேலும் குறைந்த பரிமாண நிலை பிரதிநிதித்துவ வெக்டர்களை sparse one-hot பிரதிநிதித்துவமாக மாற்றினோம். இந்த one-hot பிரதிநிதித்துவம் நினைவகத்தை திறமையாக பயன்படுத்தாது. மேலும், ஒவ்வொரு வார்த்தையும் மற்றவற்றிலிருந்து தனித்துவமாக நடத்தப்படுகிறது, எனவே one-hot குறியாக்கப்பட்ட வெக்டர்கள் வார்த்தைகளுக்கிடையிலான அர்த்த சார்ந்த ஒற்றுமைகளை வெளிப்படுத்தாது.

இந்த அலகில், **News AG** தரவுத்தொகுப்பை மேலும் ஆராய்வோம். முதலில், தரவுகளை ஏற்றவும், முந்தைய அலகிலிருந்து சில வரையறைகளைப் பெறவும்.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### எம்பெடிங் என்றால் என்ன?

**எம்பெடிங்** என்ற கருத்து என்பது வார்த்தைகளை குறைந்த பரிமாணம் கொண்ட அடர்த்தியான வெக்டர்களாக பிரதிநிதித்துவப்படுத்துவது, அவை அந்த வார்த்தையின் அர்த்தத்தை பிரதிபலிக்க வேண்டும். பொருள் கொண்ட வார்த்தை எம்பெடிங்குகளை எப்படி உருவாக்குவது என்பதை பின்னர் விவாதிப்போம், ஆனால் தற்போது எம்பெடிங்குகளை ஒரு வார்த்தை வெக்டரின் பரிமாணத்தை குறைக்கும் ஒரு முறையாகவே நினைத்துக்கொள்ளலாம்.

எம்பெடிங் லேயர் ஒரு வார்த்தையை உள்ளீடாக எடுத்து, குறிப்பிட்ட `embedding_size` அளவிலான ஒரு வெளியீடு வெக்டரை உருவாக்குகிறது. இது ஒரு வகையில் `Dense` லேயருக்கு மிகவும் ஒத்ததாக உள்ளது, ஆனால் ஒரு ஒன்-ஹாட் என்கோடட் வெக்டரை உள்ளீடாக எடுப்பதற்குப் பதிலாக, இது ஒரு வார்த்தை எண்ணை உள்ளீடாக எடுக்க முடியும்.

எங்கள் நெட்வொர்க்கில் முதல் லேயராக எம்பெடிங் லேயரை பயன்படுத்துவதன் மூலம், **bag-of-words** மாடலிலிருந்து **embedding bag** மாடலுக்கு மாற முடியும். இதில், முதலில் எங்கள் உரையில் உள்ள ஒவ்வொரு வார்த்தையையும் அதற்கான எம்பெடிங்கில் மாற்றி, பின்னர் அந்த எம்பெடிங்குகளின் மீது `sum`, `average` அல்லது `max` போன்ற ஒரு தொகுப்புக் செயல்பாட்டை கணக்கிடலாம்.

![ஐந்து வரிசை வார்த்தைகளுக்கான எம்பெடிங் வகைப்பாட்டாளரை காட்டும் படம்.](../../../../../translated_images/ta/embedding-classifier-example.b77f021a7ee67eee.webp)

எங்கள் வகைப்பாட்டாளர் நரம்பியல் நெட்வொர்க்கில் பின்வரும் லேயர்கள் உள்ளன:

* `TextVectorization` லேயர், இது ஒரு சரத்தை உள்ளீடாக எடுத்து, டோக்கன் எண்களின் டென்சரை உருவாக்குகிறது. நாம் ஒரு நியாயமான `vocab_size` சொற்கள்தொகை அளவை குறிப்பிடுவோம், மேலும் குறைவாக பயன்படுத்தப்படும் வார்த்தைகளை புறக்கணிக்கிறோம். உள்ளீடு வடிவம் 1 ஆக இருக்கும், மற்றும் வெளியீடு வடிவம் $n$ ஆக இருக்கும், ஏனெனில் $n$ டோக்கன்கள் பெறப்படும், அவற்றில் ஒவ்வொன்றும் 0 முதல் `vocab_size` வரை எண்களை கொண்டிருக்கும்.
* `Embedding` லேயர், இது $n$ எண்களை எடுத்து, ஒவ்வொரு எண்ணையும் கொடுக்கப்பட்ட நீளத்தின் அடர்த்தியான வெக்டராக குறைக்கிறது (எங்கள் உதாரணத்தில் 100). எனவே, $n$ வடிவத்திலான உள்ளீடு டென்சர் $n\times 100$ டென்சராக மாற்றப்படும்.
* தொகுப்பு லேயர், இது முதல் அச்சில் இந்த டென்சரின் சராசரியை எடுக்கும், அதாவது, இது வெவ்வேறு வார்த்தைகளுக்கு இணையான அனைத்து $n$ உள்ளீடு டென்சர்களின் சராசரியை கணக்கிடும். இந்த லேயரை செயல்படுத்த, நாம் ஒரு `Lambda` லேயரை பயன்படுத்தி, சராசரியை கணக்கிடும் செயல்பாட்டை அதில் அனுப்புவோம். வெளியீடு 100 வடிவத்தில் இருக்கும், மேலும் இது முழு உள்ளீடு வரிசையின் எண் பிரதிநிதியாக இருக்கும்.
* இறுதி `Dense` நேரியல் வகைப்பாட்டாளர்.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


`summary` அச்சிடுதலில், **output shape** பத்தியில், முதல் டென்சர் பரிமாணமான `None` மினிபேட்ச் அளவுக்கு இணையாக உள்ளது, மற்றும் இரண்டாவது டென்சர் பரிமாணம் டோக்கன் வரிசையின் நீளத்திற்கு இணையாக உள்ளது. மினிபேட்சில் உள்ள அனைத்து டோக்கன் வரிசைகளுக்கும் வெவ்வேறு நீளங்கள் உள்ளன. இதை எப்படி கையாளுவது என்பதை அடுத்த பகுதியில் விவாதிக்கப் போகிறோம்.

இப்போது நெட்வொர்க்கை பயிற்சி செய்யலாம்:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **குறிப்பு** நாம் தரவின் ஒரு துணுக்கின் அடிப்படையில் வெக்டரைசரை உருவாக்குகிறோம். இது செயல்முறையை வேகமாக்குவதற்காக செய்யப்படுகிறது, மேலும் இது எங்கள் உரையில் உள்ள அனைத்து டோக்கன்களும் சொற்களஞ்சியத்தில் இருக்காத நிலையை உருவாக்கக்கூடும். இந்த நிலைமையில், அந்த டோக்கன்கள் புறக்கணிக்கப்படும், இது சிறிது குறைந்த துல்லியத்துக்கு காரணமாக இருக்கலாம். எனினும், நிஜ வாழ்க்கையில் ஒரு உரத்தின் துணுக்கானது நல்ல சொற்களஞ்சிய மதிப்பீட்டை வழங்குகிறது.


### மாறுபடும் வரிசை அளவுகளை கையாளுதல்

மினிபேட்ச்களில் பயிற்சி எப்படி நடைபெறுகிறது என்பதை புரிந்துகொள்வோம். மேலே உள்ள உதாரணத்தில், உள்ளீட்டு டென்சரின் பரிமாணம் 1 ஆகும், மேலும் நாங்கள் 128-நீள மினிபேட்ச்களை பயன்படுத்துகிறோம், எனவே டென்சரின் உண்மையான அளவு $128 \times 1$ ஆகும். ஆனால், ஒவ்வொரு வாக்கியத்திலும் உள்ள டோக்கன்களின் எண்ணிக்கை வேறுபடுகிறது. `TextVectorization` அடுக்கு ஒரு தனி உள்ளீட்டத்திற்கு பயன்படுத்தப்பட்டால், உரை டோக்கன்களாக மாற்றப்படும் முறையைப் பொறுத்து, திரும்பும் டோக்கன்களின் எண்ணிக்கை மாறுபடும்:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


எனினும், நாம் வெக்டரைசரை பல வரிசைகளுக்கு பயன்படுத்தும்போது, அது செவ்வக வடிவத்தின் டென்சரை உருவாக்க வேண்டும், எனவே அது பயன்படுத்தப்படாத கூறுகளை PAD டோக்கனுடன் (எங்கள் வழக்கில் இது பூஜ்யம்) நிரப்புகிறது:


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

இங்கே நாங்கள் எம்பெடிங்ஸ்களை காணலாம்:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **குறிப்பு**: பூர்த்தி அளவை குறைக்க, சில சந்தர்ப்பங்களில் தரவுத்தொகுப்பில் உள்ள அனைத்து வரிசைகளையும் நீளத்தை அதிகரிக்கும் வரிசையில் (அல்லது, மேலும் துல்லியமாக, டோக்கன்களின் எண்ணிக்கையை) வரிசைப்படுத்துவது பொருத்தமாக இருக்கும். இது ஒவ்வொரு மினிபேச்சிலும் ஒரே மாதிரியான நீளத்துடன் வரிசைகள் உள்ளதைக் உறுதிப்படுத்தும்.


## Semantic embeddings: Word2Vec

முந்தைய உதாரணத்தில், embedding layer வார்த்தைகளை வெக்டார் பிரதிநிதிகளாக மாற்ற கற்றுக்கொண்டது, ஆனால் இந்த பிரதிநிதிகளுக்கு அர்த்தபூர்வமான பொருள் இல்லை. ஒரே மாதிரியான வார்த்தைகள் அல்லது ஒத்த வார்த்தைகள் வெக்டார் தூரத்தின் அடிப்படையில் (உதாரணமாக euclidian distance) ஒன்றுக்கொன்று அருகில் இருக்கும் வகையில் ஒரு வெக்டார் பிரதிநிதியை கற்றுக்கொள்வது நல்லது.

அதற்காக, [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) போன்ற ஒரு தொழில்நுட்பத்தைப் பயன்படுத்தி, ஒரு பெரிய உரைத் தொகுப்பில் embedding மாடலை முன்கூட்டியே பயிற்சி செய்ய வேண்டும். இது வார்த்தைகளின் பகிர்ந்த பிரதிநிதியை உருவாக்க இரண்டு முக்கிய கட்டமைப்புகளை அடிப்படையாகக் கொண்டது:

 - **Continuous bag-of-words** (CBoW), இதில் சுற்றியுள்ள சூழலிலிருந்து ஒரு வார்த்தையை கணிக்க மாடலை பயிற்சி செய்கிறோம். ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$ கொடுக்கப்பட்டால், மாடலின் நோக்கம் $(W_{-2},W_{-1},W_1,W_2)$-இல் இருந்து $W_0$-ஐ கணிக்க வேண்டும்.
 - **Continuous skip-gram** என்பது CBoW-க்கு எதிர்மாறானது. மாடல் தற்போதைய வார்த்தையை கணிக்க சூழலின் சாளரத்தில் உள்ள வார்த்தைகளைப் பயன்படுத்துகிறது.

CBoW வேகமாக செயல்படுகிறது, ஆனால் skip-gram மெதுவாக இருந்தாலும், அரிதான வார்த்தைகளை பிரதிநிதித்துவப்படுத்துவதில் சிறப்பாக செயல்படுகிறது.

![CBoW மற்றும் Skip-Gram அல்காரிதங்களை வார்த்தைகளை வெக்டார்களாக மாற்றும் முறையை காட்டும் படம்.](../../../../../translated_images/ta/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Google News dataset-ல் முன்கூட்டியே பயிற்சி செய்யப்பட்ட Word2Vec embedding-ஐ பரிசோதிக்க, **gensim** நூலகத்தைப் பயன்படுத்தலாம். கீழே 'neural' என்ற வார்த்தைக்கு மிகவும் ஒத்த வார்த்தைகளை காணலாம்.

> **குறிப்பு:** முதன்முதலில் வார்த்தை வெக்டார்களை உருவாக்கும்போது, அவற்றை பதிவிறக்கம் செய்வது சில நேரம் எடுக்கலாம்!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


நாம் வார்த்தையிலிருந்து வெக்டர் எம்பெடிங்கையும் எடுக்கலாம், இது வகைப்பாடு மாடலைப் பயிற்சியில் பயன்படுத்தப்படுகிறது. எம்பெடிங்கில் 300 கூறுகள் உள்ளன, ஆனால் தெளிவுக்காக இங்கு வெக்டர் முதல் 20 கூறுகளை மட்டுமே காட்டுகிறோம்:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

செயல்முறை உள்ளடக்கங்களைப் பற்றிய சிறந்த விஷயம் என்னவென்றால், நீங்கள் அர்த்தத்தின் அடிப்படையில் வெக்டர் குறியீட்டை மாற்ற முடியும். உதாரணமாக, *king* மற்றும் *woman* என்ற சொற்களுக்கு όσοவும் அருகிலுள்ள மற்றும் *man* என்ற சொல்லுக்கு όσοவும் தூரமாக உள்ள ஒரு சொல்லை கண்டுபிடிக்க கேட்கலாம்:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

மேலே உள்ள உதாரணம் சில உள்நாட்டு GenSym மந்திரங்களை பயன்படுத்துகிறது, ஆனால் அடிப்படை தர்க்கம் மிகவும் எளிமையானது. எம்பெடிங்க்ஸ் பற்றிய ஒரு சுவாரஸ்யமான விஷயம் என்னவென்றால், நீங்கள் எம்பெடிங் வெக்டர்களில் சாதாரண வெக்டர் செயல்பாடுகளை செய்யலாம், மேலும் அது சொற்களின் **அர்த்தங்கள்** மீது செயல்பாடுகளை பிரதிபலிக்கும். மேலே உள்ள உதாரணத்தை வெக்டர் செயல்பாடுகளின் அடிப்படையில் விவரிக்கலாம்: **KING-MAN+WOMAN** என்பதற்கான வெக்டரை கணக்கிடுகிறோம் (செயல்பாடுகள் `+` மற்றும் `-` தொடர்புடைய சொற்களின் வெக்டர் பிரதிநிதித்துவங்களில் செய்யப்படுகின்றன), பின்னர் அந்த வெக்டருக்கு அடுத்ததாக அகராதியில் உள்ள மிக அருகிலான சொல்லை கண்டுபிடிக்கிறோம்:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: *man* மற்றும் *woman* வெக்டர்களுக்கு சிறிய குணகங்களைச் சேர்க்க வேண்டியிருந்தது - அவற்றை நீக்கி என்ன நடக்கிறது என்பதைப் பாருங்கள்.

நெருக்கமான வெக்டரை கண்டுபிடிக்க, நாங்கள் TensorFlow இயந்திரத்தைப் பயன்படுத்தி, எங்கள் வெக்டர் மற்றும் அகராதியில் உள்ள அனைத்து வெக்டர்களுக்கும் இடையிலான தூரங்களின் வெக்டரை கணக்கிடுகிறோம், பின்னர் `argmin` ஐப் பயன்படுத்தி குறைந்தபட்சமான வார்த்தையின் குறியீட்டை கண்டுபிடிக்கிறோம்.


Word2Vec வார்த்தை அர்த்தங்களை வெளிப்படுத்த ஒரு சிறந்த வழியாக தோன்றினாலும், இதற்கு பல குறைகள் உள்ளன, அவற்றில் சில:

* CBoW மற்றும் skip-gram மாதிரிகள் இரண்டும் **predictive embeddings** ஆகும், மேலும் அவை உள்ளூர் சூழலை மட்டுமே கணக்கில் எடுத்துக்கொள்கின்றன. Word2Vec உலகளாவிய சூழலின் நன்மைகளை பயன்படுத்துவதில்லை.
* Word2Vec வார்த்தையின் **morphology**-யை, அதாவது ஒரு வார்த்தையின் அர்த்தம் அதன் வேர் போன்ற பல பகுதிகளின் அடிப்படையில் மாறக்கூடும் என்பதை கணக்கில் எடுத்துக்கொள்வதில்லை.

**FastText** இரண்டாவது குறையை சரிசெய்ய முயற்சிக்கிறது, மேலும் Word2Vec-ஐ அடிப்படையாகக் கொண்டு ஒவ்வொரு வார்த்தைக்கும் மற்றும் ஒவ்வொரு வார்த்தையின் உள்ளே உள்ள character n-grams-க்கு வெக்டர் பிரதிநிதிகளை கற்றுக்கொள்கிறது. இந்த பிரதிநிதிகளின் மதிப்புகள் ஒவ்வொரு பயிற்சி படியில் ஒரு வெக்டராக சராசரியாக்கப்படுகின்றன. இது முன்பயிற்சிக்கு கூடுதல் கணினி செயல்பாடுகளை சேர்த்தாலும், sub-word தகவல்களை encode செய்ய வார்த்தை embeddings-ஐ சாத்தியமாக்குகிறது.

மற்றொரு முறை, **GloVe**, வார்த்தை embeddings-க்கு ஒரு மாறுபட்ட அணுகுமுறையை பயன்படுத்துகிறது, இது வார்த்தை-சூழல் மேட்ரிக்ஸின் factorization அடிப்படையில் உள்ளது. முதலில், இது பல்வேறு சூழல்களில் வார்த்தை நிகழ்வுகளின் எண்ணிக்கையை எண்ணும் ஒரு பெரிய மேட்ரிக்ஸை உருவாக்குகிறது, பின்னர் இந்த மேட்ரிக்ஸை குறைந்த பரிமாணங்களில் பிரதிநிதித்துவப்படுத்த முயற்சிக்கிறது, reconstruction loss-ஐ குறைக்குமாறு.

gensim நூலகம் இந்த வார்த்தை embeddings-ஐ ஆதரிக்கிறது, மேலும் மேலே உள்ள மாதிரி ஏற்றும் குறியீட்டை மாற்றுவதன் மூலம் அவற்றுடன் நீங்கள் பரிசோதிக்கலாம்.


## Keras-ல் முன்பே பயிற்சி செய்யப்பட்ட எம்பெடிங்க்களை பயன்படுத்துவது

மேலே உள்ள உதாரணத்தை மாற்றி, எம்பெடிங் லேயரில் உள்ள மாட்ரிக்ஸை Word2Vec போன்ற அர்த்தபூர்வமான எம்பெடிங்க்களால் முன்பே நிரப்பலாம். முன்பே பயிற்சி செய்யப்பட்ட எம்பெடிங் மற்றும் உரை தொகுப்பின் சொற்களஞ்சியங்கள் (vocabularies) பொதுவாக பொருந்தாது, எனவே ஒன்றை தேர்வு செய்ய வேண்டும். இங்கே இரண்டு சாத்தியமான விருப்பங்களை ஆராய்கிறோம்: Tokenizer சொற்களஞ்சியத்தை பயன்படுத்துவது மற்றும் Word2Vec எம்பெடிங்க்களின் சொற்களஞ்சியத்தை பயன்படுத்துவது.

### Tokenizer சொற்களஞ்சியத்தை பயன்படுத்துவது

Tokenizer சொற்களஞ்சியத்தை பயன்படுத்தும்போது, சொற்களஞ்சியத்தில் உள்ள சில சொற்களுக்கு Word2Vec எம்பெடிங்க்கள் இருக்கும், சிலவற்றுக்கு இருக்காது. எங்கள் சொற்களஞ்சியத்தின் அளவு `vocab_size` என்றும், Word2Vec எம்பெடிங் வெக்டர் நீளம் `embed_size` என்றும் கொடுக்கப்பட்டால், எம்பெடிங் லேயர் `vocab_size`$\times$`embed_size` வடிவத்தில் ஒரு எடை மாட்ரிக்ஸால் பிரதிநிதித்துவம் செய்யப்படும். இந்த மாட்ரிக்ஸை சொற்களஞ்சியத்தைச் சென்று நிரப்புவோம்:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Word2Vec அகராதியில் இல்லாத வார்த்தைகளுக்கு, அவற்றை பூஜ்யமாகவோ அல்லது ஒரு சீரற்ற வெக்டரை உருவாக்குவதோடு விடலாம்.

இப்போது, முன்பே பயிற்சியளிக்கப்பட்ட எடைகளுடன் ஒரு embedding layer-ஐ வரையறுக்கலாம்:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

இப்போது நமது மாதிரியை பயிற்சி செய்யலாம்.


In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Note**: `Embedding` உருவாக்கும்போது `trainable=False` என அமைத்துள்ளோம், இதனால் Embedding அடுக்கு மீண்டும் பயிற்சி செய்யப்படாது. இது துல்லியத்தை சிறிது குறைக்கக்கூடும், ஆனால் பயிற்சியின் வேகத்தை அதிகரிக்கிறது.

### எம்பெடிங் சொற்களஞ்சியத்தை பயன்படுத்துதல்

முந்தைய அணுகுமுறையில் ஒரு சிக்கல் என்னவென்றால், TextVectorization மற்றும் Embedding பயன்படுத்தும் சொற்களஞ்சியங்கள் வேறுபட்டவை. இந்த பிரச்சினையைத் தீர்க்க, நாம் பின்வரும் தீர்வுகளில் ஒன்றைப் பயன்படுத்தலாம்:
* Word2Vec மாதிரியை எங்கள் சொற்களஞ்சியத்தில் மீண்டும் பயிற்சி செய்யவும்.
* முன்பே பயிற்சி செய்யப்பட்ட Word2Vec மாதிரியின் சொற்களஞ்சியத்துடன் எங்கள் தரவுத்தொகுப்பை ஏற்றவும். தரவுத்தொகுப்பை ஏற்றும்போது பயன்படுத்தப்படும் சொற்களஞ்சியங்களை ஏற்றும்போது குறிப்பிடலாம்.

இரண்டாவது அணுகுமுறை எளிதாகத் தோன்றுகிறது, எனவே அதை செயல்படுத்துவோம். முதலில், Word2Vec எம்பெடிங்கில் இருந்து எடுத்துக்கொள்ளப்பட்ட குறிப்பிட்ட சொற்களஞ்சியத்துடன் ஒரு `TextVectorization` அடுக்கை உருவாக்குவோம்:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

ஜென்சிம் வார்த்தை எம்பெடிங்ஸ் நூலகத்தில் `get_keras_embeddings` என்ற வசதியான செயல்பாடு உள்ளது, இது தானாகவே உங்களுக்கு பொருந்தும் கேராஸ் எம்பெடிங்ஸ் லேயரை உருவாக்கும்.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


எங்கள் தரவுத்தொகுப்பில் உள்ள சில வார்த்தைகள் முன்பே பயிற்சி அளிக்கப்பட்ட GloVe சொற்களஞ்சியத்தில் இல்லை என்பதால், அவை அடிப்படையாகவே புறக்கணிக்கப்படுகின்றன. இதனால், அதிக துல்லியத்தை காண முடியவில்லை. இதை சமாளிக்க, எங்கள் தரவுத்தொகுப்பின் அடிப்படையில் நாங்கள் சொந்தமாக எம்பெடிங்குகளை பயிற்சி அளிக்கலாம்.


## சூழலியல் எம்பெடிங்குகள்

Word2Vec போன்ற பாரம்பரிய முன்பதிவுசெய்யப்பட்ட எம்பெடிங் பிரதிநிதிகளின் முக்கிய குறைபாடுகளில் ஒன்று, ஒரு வார்த்தையின் சில அர்த்தங்களைப் பிடிக்க முடிந்தாலும், வெவ்வேறு அர்த்தங்களை வேறுபடுத்த முடியாது என்பதுதான். இது பின்னர் மாடல்களில் சிக்கல்களை ஏற்படுத்தலாம்.

உதாரணமாக, 'play' என்ற வார்த்தைக்கு இந்த இரண்டு வாக்கியங்களில் வெவ்வேறு அர்த்தங்கள் உள்ளன:
- நான் தியேட்டரில் ஒரு **play**-ஐப் பார்த்தேன்.
- ஜான் தனது நண்பர்களுடன் **play** செய்ய விரும்புகிறார்.

நாம் பேசிய முன்பதிவுசெய்யப்பட்ட எம்பெடிங்குகள் 'play' என்ற வார்த்தையின் இரு அர்த்தங்களையும் ஒரே எம்பெடிங்கில் பிரதிநிதித்துவப்படுத்துகின்றன. இந்தக் குறையைத் தாண்ட, **மொழி மாடல்** அடிப்படையில் எம்பெடிங்குகளை உருவாக்க வேண்டும், இது ஒரு பெரிய உரை தொகுப்பில் பயிற்சி பெறுகிறது மற்றும் வார்த்தைகள் வெவ்வேறு சூழல்களில் எப்படி சேர்க்கப்படலாம் என்பதை *அறிகிறது*. சூழலியல் எம்பெடிங்குகளைப் பற்றி விவாதிப்பது இந்த பாடத்திட்டத்தின் வரம்புக்கு வெளியே உள்ளது, ஆனால் மொழி மாடல்களைப் பற்றி பேசும் அடுத்த அலகில் நாம் அவற்றை மீண்டும் பார்க்கப் போகிறோம்.



---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கின்றோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளுங்கள். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
